In [5]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
%%sql
sqlite:///smaller.db
ATTACH DATABASE 'NG.db' AS NG;
create table NG.data as select * from data where fuel_actual is 'NG';

Done.
Done.


[]

In [6]:
%%sql
sqlite:///NG.db
drop table if exists cf;
create table cf as 
select name, d.orispl_code, d.unitid, state, op_date, op_hour, gload, 
cast(HEAT_INPUT as float) / cast(max_gen_hr as float) as capacity_factor_hr,
cast(gload as float) / cast(max_gen_gl as float) as capacity_factor_gl
from (select orispl_code, unitid, max(CAST(HEAT_INPUT as float)) as max_gen_hr,
      max(CAST(gload as float)) as max_gen_gl
from data group by orispl_code, unitid) as m, data as d
where d.orispl_code = m.orispl_code and d.unitid = m.unitid;
select * from cf limit 5;

Done.
Done.
Done.


name,orispl_code,unitid,state,op_date,op_hour,gload,capacity_factor_hr,capacity_factor_gl
River Road,7605,1,WA,01-01-2001,0,258.0,0.916360481731,0.945054945055
River Road,7605,1,WA,01-01-2001,1,258.0,0.918758930394,0.945054945055
River Road,7605,1,WA,01-01-2001,2,258.0,0.921004286589,0.945054945055
River Road,7605,1,WA,01-01-2001,3,259.0,0.922790365381,0.948717948718
River Road,7605,1,WA,01-01-2001,4,259.0,0.920085731782,0.948717948718


In [7]:
import sqlite3
import pandas
import os.path
con = sqlite3.connect('NG.db')
labels = pandas.read_sql("select orispl_code, unitid from cf group by orispl_code, unitid", con)


In [8]:
# Generates csv file for each unit, for each year, of capacity factors.
if not os.path.exists('cf'):
    os.makedirs('cf')
for i, row in labels.iterrows():
    print row['orispl_code'], row['unitid']
    for year in range(2001, 2017):
        file_name = 'cf/' + str(row['orispl_code']) + '_' + row['unitid'].replace('*', '') + '_' + str(year) + '.csv'
        if os.path.isfile(file_name): continue
        table = pandas.read_sql("select name, orispl_code, unitid, op_date, op_hour, capacity_factor_gl, capacity_factor_hr from cf\
        where orispl_code is " + str(row['orispl_code']) + " and unitid is '" + row['unitid'] + "' and \
        substr(op_date, -4) is '" + str(year) + "'", con)
        table.to_csv(file_name)

9 CTG-1
116 1
116 2
117 CC4
117 CC5A
117 CC5B
118 1
118 2
118 CT3
120 1
120 CT5
120 CT6
124 GT1
126 1
126 2
126 3
141 1
141 2
141 3
147 K-1
147 K-2
147 K-7
160 1
160 4
228 10
228 9
246 1
246 2
247 3
247 4
247 5
247 6
247 7
259 1
259 2
259 3
259 4
260 1A
260 2A
260 3A
260 4A
260 6-1
260 7-1
271 1
271 2
271 3
271 4
271 5
271 6
271 7
273 3-1
302 1
302 2
302 3
302 4
302 5
310 1
310 2
310 3
310 4
315 1
315 2
315 3
315 4
315 5
315 6
329 1
329 2
329 31
329 32
329 41
329 42
330 3
330 4
330 5
330 7
331 1
331 2
331 3
331 4
334 1
334 2
334 3
334 4
335 1
335 2
335 3A
335 4A
341 1
341 2
341 3
341 4
345 1
345 2
350 1
350 2
356 17
356 5
356 6
356 7
356 8
358 1
358 2
358 3-1
358 3-2
358 4-1
358 4-2
375 M4
377 4
377 5
377 9
389 2-2
389 3
389 3-1
389 3-2
389 4
399 **10A
399 **10B
399 10
399 11
399 12
399 13
399 14
400 1
400 10
400 11
400 12
400 13
400 14
400 15
400 16
400 2
400 3
400 4
400 5
400 6
400 9
404 1
404 2
404 3
404 4
404 6
404 7
408 3
408 4
408 5
408 6
408 7
420 B1
420 B2
420 B3
422 GT3
422 GT